In [ ]:
！记得补充对应英文

In [ ]:
AI > Machine Learning > Deep Learning

Machine learning algorithm:
    1. Decision Tree
    2. Random Forest
    3. XGBoost
    4. Support Vector Machine

#### 1. Decision Tree 单一树状结构

In [ ]:
Main idea: 基于features递归分割数据，分裂标准有信息增益和基尼系数（？具体区别）

#### 2. Random Forest 多棵决策树，集成模型

In [ ]:
Main idea: 对原始数据有放回地抽样（bootstrap），产生与原始数据集大小相同的子集，每个样本被抽到的概率是 p=1-((n-1)/n)^n=63.2%
           当n足够大时（>=1000），意味着每个树的样本中都有63.2%的独特样本，每两棵树之间有39.9%的重复样本，只在一棵树的样本为46.5%，两棵树都没有的样本为13.5%
           当n比较小时，如n=50，这一比例会变小
           以这种抽样方法构造出很多子集，以此训练对应数量的决策树，这些决策树之间是弱相关的（还是没懂为什么弱相关）
           最后根据这么多棵决策树的共同结果给出决策，分类用众数mode，回归取平均
           补充：（1）存在 out of bag samples袋外样本，可以用于验证
                 （2）确定单棵树的样本后会进行特征抽样feature bagging，随机森林模型中包含max_features参数，该参数常见取值有auto/sqrt/log2等

#### 3. XGBoost 集成模型

In [ ]:
Main idea: 按顺序训练多棵树，每个树学习前一棵树的残差，通过步步逼近防止过拟合
           优化目标：损失函数 + 正则项（防过拟合）

#### 4. Support Vector Machine 几何分割

In [ ]:
Main idea: 寻找最大化分类间隔的超平面
           核技巧：通过核函数处理非线性分割，核函数也有好几种

### Code
#### 1. 数据处理

In [ ]:
# data
import numpy as np
import pandas as pd

# picture
import matplotlib.pyplot as plt
import seaborn as sns

# 绘图代码执行后立刻显示在代码框下
%matplotlib inline

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split

# 只返回分数
from sklearn.model_selection import cross_val_score
# 还返回如响应时间等其他数据，字典格式
from sklearn.model_selection import cross_validate

# 不同调参方式
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, ConfusionMatrixDisplay
from sklearn.metrics import plot_confusion_matrix

# 决策树需要
from sklearn.tree import DecisionTreeClassifier 
from sklearn import tree

# 随机森林需要
from sklearn.ensemble import RandomForestClassifier

# XGBoost需要
import xgboost as xgb

# SVM需要
from sklearn.svm import SVC

import warnings
warnings.simplefilter('ignore')
warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore',category=DeprecationWarning)


In [ ]:
# Import the dataset

# 逗号分隔：sep=','，pd.read_table可以自定义分隔符

fires = pd.read_table("C:/Users/ruhan/Desktop/Classes/2025 ML and NLP/Week 2/Algerian_forest_fires_dataset.csv", sep=',')

In [ ]:
# Examine data

type(fires)
fires.shape
fires.columns
fires.head()
fires.info()
fires.describe()
fires.isnull().sum()

# Examine categorical variables 
cat_vars = ['PD',"Classes"]
for col in cat_vars:
    print(fires[col].value_counts())
    
# Modify data

fires = fires.drop(['day','month','year'],axis=1)
fires["Classes"].unique()
fires["Classes"] = fires["Classes"].str.strip()
fires= pd.get_dummies(fires, columns = ['Classes'])
fires["Classes"].value_counts()
sns.countplot(fires["Classes"], label = "Count")
plt.figure(figsize=(10, 8))
sns.scatterplot(x = 'Temperature', y = 'FFMC', hue = 'Classes', data = fires)

# Let's check the correlation between the variables 
plt.figure(figsize=(20,10)) 
sns.heatmap(fires.corr(), annot=True) 

# Drop BUI variable with less interpretability and high correlation 
fires = fires.drop(['BUI'],axis=1)

# Handle categorial variables 
ord_enco = OrdinalEncoder(categories=[['Low ', 'Medium', 'High']])
en_PD = ord_enco.fit_transform(fires.loc[:, ['PD']]) #
en_PD = pd.DataFrame(en_PD)
en_PD  = en_PD.rename(columns= {0:'PDnew'})
fires['PDnew'] = ord_enco.fit_transform(fires.loc[:, ['PD']])


In [ ]:
# Split dataset into features and target

target = fires['Classes']
features = fires.drop('Classes', axis=1)

# split into test and train set
X_train,X_test,y_train,y_test = train_test_split(features,target,test_size=0.2,random_state=42)
X_train.shape,X_test.shape

#### 2. 初始化各分类器 Initialize the classifier with default parameters

In [ ]:
# Decision tree
# if we only use default, no hyperparameter tuning or cross validation
Classifier_DT = DecisionTreeClassifier(random_state=42)
Classifier_DT.get_params()

# Random forest
Classifier_RF = RandomForestClassifier(random_state=42)
Classifier_RF = RandomForestClassifier(n_estimators=2,random_state=1)
Classifier_RF.get_params()
RandomForestClassifier().get_params()

# XGBoost
Classifier_Xgb = xgb.XGBClassifier(random_state=42)
Classifier_Xgb = xgb.XGBClassifier(n_estimators=100, objective='binary:logistic', eta=0.1, max_depth=3)
# method 1 查看定义好的包含部分参数的
Classifier_Xgb.get_params()
# method 2 查看最原始的没有任何参数的格式
xgb.XGBClassifier().get_params()

# Support Vector Machine
Classifier_SVC= SVC()
Classifier_SVC.get_params()
Classifier_SVC.get_params()
SVC().get_params()

# Train the classifier
Classifier_DT.fit(X_train, y_train)
Classifier_RF.fit(X_train, y_train)
Classifier_Xgb.fit(X_train, y_train)
# SVC训练前特征必须标准化
Classifier_SVC.fit(X_train, y_train)

# calculate cross validation score 
cv_results_DT = cross_val_score(Classifier_DT, X_train, y_train, cv=5)
cv_results_RF = cross_val_score(Classifier_RF, X_train, y_train, cv=5)
cv_results_Xgb = cross_val_score(Classifier_Xgb, X_train, y_train, cv=5)
cv_results_SVC = cross_val_score(Classifier_SVC, X_train, y_train, cv=5)


# Evaluate 评估
# Decision tree
print(classification_report(y_test, Classifier_DT.predict(X_test)))
# Create confusion matrix
print(confusion_matrix(y_test,Classifier_DT.predict(X_test)))

# Visualize the tree
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 6))
tree.plot_tree(clf, feature_names=X_train.columns, class_names=clf.classes_, filled=True)
plt.show()

#  Random forest
y_predict = Classifier_RF.predict(X_test)
print('Accuracy: %.3f' % accuracy_score(np.array(y_test), y_predict))

#### 2.1  Hyperparameter Tuning

##### 2.1.1 Decision tree

In [ ]:
# Maximum depth of the tree
max_depth = [None, 5, 10, 20]

# Minimum number of samples required to split an internal node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at a leaf node
min_samples_leaf = [1, 2, 4]

# Criterion for measuring quality of split
criterion = ['gini', 'entropy']

# Create a random grid
rnd_grid = {
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
    'criterion': criterion
}

print(rnd_grid)

##### 2.1.2 Random forest

In [ ]:
# number of trees 
n_estimator = [int(x) for x in np.linspace(start =200, stop=1000,num=10)]
# number of feature at each split
max_features = ['sqrt']
# max depth 
max_depth = [int(x) for x in np.linspace(start =10, stop=100,num=10)]
# min number of sample at each split 
min_sam_split = [2,5,10]


# Create a random grid 
rnd_grid= {'n_estimators':n_estimator,
          'max_features':max_features,
          'max_depth' :max_depth,
          'min_samples_split' :min_sam_split}

print(rnd_grid)

# Use randomizedsearchcv
Classifier_RF_random = RandomizedSearchCV(estimator=Classifier_RF,param_distributions =rnd_grid,
                               n_iter = 10,cv=3,verbose=2,random_state=33,n_jobs=-1,return_train_score=True)
Classifier_RF_random.fit(X_train, y_train)

# Examine the results from each combination of hyperparameters 
pd.DataFrame(Classifier_RF_random.cv_results_)

print('Best hyper parameter:', Classifier_RF_random.best_params_)

# Evaluate
best_random = Classifier_RF_random.best_estimator_
best_random.fit(X_train,y_train)
y_pred_rnd = best_random.predict(X_test)
rnd_accuracy = accuracy_score(np.array(y_test), y_pred_rnd)

print('Random best hyperparameter accuracy: ',rnd_accuracy)

cm = confusion_matrix(y_test, y_pred_rnd)
cm

# View fature scores 
feature_score = pd.Series(best_random.feature_importances_,index=X_train.columns).sort_values(ascending =False)
feature_score

# Visualize feature importance 
sns.barplot(x=feature_score,y=feature_score.index)
plt.xlabel("Feature importance")
plt.ylabel("Feature name")

##### 2.1.3 XGBoost

In [ ]:
# max depth 
max_depth = [3,6,10]
# Learning rate  
learning_rate = [0.01,0.1]
# number of trees 
n_estimator = [50,100]
# col sample by tree 
col_sample = [0.6,1]


# Create a random grid 
rnd_grid= {'n_estimators':n_estimator,
          'learning_rate':learning_rate,
          'max_depth' :max_depth,
          'colsample_bytree' :col_sample}

print(rnd_grid)

# Use randomizedsearchcv
Classifier_Xgb_random = RandomizedSearchCV(estimator=Classifier_Xgb,param_distributions =rnd_grid,
                               n_iter = 10,cv=3,verbose=2,random_state=33,n_jobs=-1,return_train_score=True)
Classifier_Xgb_random.fit(X_train, y_train)

# Examine the results from each combination of hyperparameters 
pd.DataFrame(Classifier_Xgb_random.cv_results_)

print('Best hyper parameter:', Classifier_Xgb_random.best_params_)

# Evaluate 
best_random = Classifier_Xgb_random.best_estimator_
best_random.fit(X_train, y_train)
y_pred_best = best_random.predict(X_test)
rnd_accuracy = accuracy_score(np.array(y_test), y_pred_best)

print('Random best hyperparameter accuracy: ',rnd_accuracy)

# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred_best)
cm

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=best_random.classes_)
disp.plot(cmap='Blues')
plt.title("Confusion Matrix - Test Set")
plt.show()

# View fature scores 
feature_score = pd.Series( best_random.feature_importances_,index=X_train.columns).sort_values(ascending =False)
feature_score

# Visualize feature importance 
sns.barplot(x=feature_score,y=feature_score.index)
plt.xlabel("Feature importance")
plt.ylabel("Feature name")


##### 2.1.4 SVM
先特征标准化再训练调参

In [ ]:
# 标准化
scaler = StandardScaler()
cols = X_train.columns

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = pd.DataFrame(X_train, columns=[cols])
X_test = pd.DataFrame(X_test, columns=[cols])

X_train.describe()

In [ ]:
# kernel type
kernel = ['linear','rbf']

# C parameter 
C = [0.1,1,10]


# Create a grid 
h_params= {'kernel':kernel,
          'C':C}

print(h_params)

# Use grid search
Classifier_SVC_grid=GridSearchCV(Classifier_SVC,h_params,cv=5)
Classifier_SVC_grid.fit(X_train, y_train)

# Examine the results from each combination of hyperparameters 
pd.DataFrame(Classifier_SVC_grid.cv_results_)
print('Best hyper parameter:', Classifier_SVC_grid.best_params_)

# Evaluate 
best_grid = Classifier_SVC_grid.best_estimator_
best_grid.fit(X_train, y_train)
y_pred_best = best_grid.predict(X_test)
rnd_accuracy = accuracy_score(np.array(y_test), y_pred_best)

print('Grid best hyperparameter accuracy: ',rnd_accuracy)

# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred_best)
cm

plot_confusion_matrix(svc_best, X_test, y_test)  
plt.show()

# View fature scores 
# linear
pd.Series(abs(svc_best.coef_[0]),index=X_train.columns).sort_values(ascending =True).plot(kind='barh')
